In [2]:
# Baixa e extrai os arquivos do Portal da Transparência
import requests, zipfile, io, pathlib

def download_set(name):
    for month in range(1, 13):
        path = f'./dataset/{name}/{month}'
        pathlib.Path(path).mkdir(parents=True, exist_ok=True)
        req = requests.get(f'http://transparencia.gov.br/download-de-dados/{name}/2020{month:02}', stream=True)
        file = zipfile.ZipFile(io.BytesIO(req.content))
        file.extractall(path)

download_set('compras')
download_set('licitacoes')

In [118]:
# Setup do banco de dados
import sqlite3, glob, csv

db = sqlite3.connect('DB.sqlite')
with open('schema.sql', 'r') as schema:
    db.executescript(schema.read())

def insert_table(data, name, *headers):
    """Insere todos os valores dos headers em uma tabela"""
    db.executemany(f"INSERT INTO {name} VALUES ({','.join('?'*len(headers))}) ON CONFLICT DO NOTHING;", [tuple(r[key] for key in headers) for r in data])

def convert_col_real(table, col):
    db.execute(f"UPDATE {table} SET {col} = CAST(REPLACE({col}, ',', '.') AS REAL)")

def read_all_csv(pattern):
    for filename in glob.iglob(f"dataset/licitacoes/**/*_{pattern}.csv"):
        with open(filename, mode='r', encoding='iso-8859-1', newline='') as file:
            yield list(csv.DictReader(file, quotechar='"', delimiter=';'))

for r in read_all_csv('Licitaç╞o'):
    insert_table(r, 'unidade_gestora', 'Código UG', 'Nome UG')
    insert_table(r, 'modalidade_compra', 'Código Modalidade Compra', 'Modalidade Compra')
    insert_table(r, 'orgao_superior', 'Código Órgão Superior', 'Nome Órgão Superior')
    insert_table(r, 'orgao_subordinado', 'Código Órgão', 'Nome Órgão', 'Código Órgão Superior')
    insert_table(r, 'licitacao', 'Número Licitação', 'Código Modalidade Compra', 'Número Processo', 'Objeto', 'Situação Licitação', 'Código UG', 'Código Órgão', 'Município', 'Data Resultado Compra', 'Data Abertura', 'Valor Licitação')
convert_col_real('licitacao', 'valor')

for r in read_all_csv('ParticipantesLicitaç╞o'):
    insert_table(r, 'pessoa_juridica', 'CNPJ Participante', 'Nome Participante')
    insert_table(r, 'participante_licitacao', 'Número Licitação', 'Código Modalidade Compra', 'Número Processo', 'Código UG', 'Código Órgão', 'Descrição Item Compra', 'CNPJ Participante', 'Flag Vencedor')

for r in read_all_csv('ItemLicitaç╞o'):
    insert_table(r, 'pessoa_juridica', 'CNPJ Vencedor', 'Nome Vencedor')
    insert_table(r, 'item_licitacao', 'Número Licitação', 'Código Modalidade Compra', 'Número Processo', 'Código UG', 'Código Órgão', 'Descrição', 'CNPJ Vencedor', 'Quantidade Item', 'Valor Item')
convert_col_real('item_licitacao', 'valor_item')
    
db.commit()
db.close()